In [12]:
import os
import sys
module_path = os.path.abspath(os.path.join("../.."))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from jupyter_notebook import load_parameters 

In [14]:
import pandas as pd

pars = load_parameters()

input_file = pars.get('input')
output_file = pars.get('output')

input_col = pars.get('input_col')
output_col = pars.get('output_col')

debug = pars.get('debug')

In [23]:
dataset = pd.read_pickle(input_file)

#### Process In-Out Terminal-Like Code
The goal of this function is to remove tags related ot notebook or terminal-like codes

Example:
```python
In [11]: df
df.append()
Out[11]: 
A B C D 
2000-01-03 -0.59885 -0.18141 -0.68828 -0.77572
2000-01-04 0.83935 0.15993 0.95911 -1.12959

In [12]: df[(df.values > 1.5).any(1)]
Out[12]: 
A B C D 
2000-01-05 2.8021 -0.1086 -1.62114 -0.2017
2000-01-06 0.7167 -0.2671 1.36029 1.7425
```

Should extract only:
```python
df
df.append()
df[(df.values > 1.5).any(1)]
```

#### Process Mixed In-Out and In-In Code

``` python
In [2]: st_time = time.time(); initialload = sm.iolib.genfromdta("/home/myfile.dta"); ed_time = time.time(); print (ed_time - st_time)


In [2]: st_time = time.time(); initialload = sm.iolib.genfromdta("/home/myfile.dta"); ed_time = time.time(); print (ed_time - st_time)
666.523324013

In [3]: type(initialload)
Out[3]: numpy.ndarray

In [4]: initialload.shape

Out[4]: (4809584,)
```


In [89]:
import re

terminal_out_in_regex = "(Out\[.+?\]:.*?In \[.+?\]:)"
sequence_out = re.compile(terminal_out_in_regex, re.DOTALL)

terminal_in_regex = "(In \[.+?\]:)"
sequence_in = re.compile(terminal_in_regex, re.DOTALL)

terminal_out_regex = "(Out\[.+?\]: )"

def preprocessTerminalLikeCode(code):    
    pure_code = re.sub(sequence_out, '', code)
    pure_code = re.sub(sequence_in, '', pure_code)
    return pure_code

# def preprocessTerminalLikeCode(code):    
#     pure_code = re.sub(terminal_out_regex, '', code)
#     return pure_code
    

def preprocessCode(code):
    # First Method for pre-processing
    terminalLikeCode = preprocessTerminalLikeCode(code)
    if terminalLikeCode:
        code = ''.join(terminalLikeCode)
    return code
    
def preprocessPost(post, col):
    strcode = str(post[col])
    # Get the list of codes
    post[output_col] = preprocessCode(strcode)
    return post

processed_dataset = dataset.apply(preprocessPost, axis=1, args=(input_col,))

In [90]:
# text = """
# In [2]: st_time = time.time(); initialload = sm.iolib.genfromdta("/home/myfile.dta"); ed_time = time.time(); print (ed_time - st_time)

# In [2]: st_time = time.time(); initialload = sm.iolib.genfromdta("/home/myfile.dta"); ed_time = time.time(); print (ed_time - st_time)

# In [3]: type(initialload)
# Out[3]: numpy.ndarray

# In [4]: initialload.shape

# Out[4]: (4809584,)
# """

# print(text)

# print(preprocessTerminalLikeCode(text))


st_time = time.time(); initialload = sm.iolib.genfromdta("/home/myfile.dta"); ed_time = time.time(); print (ed_time - st_time)

st_time = time.time(); initialload = sm.iolib.genfromdta("/home/myfile.dta"); ed_time = time.time(); print (ed_time - st_time)

type(initialload)
numpy.ndarray

initialload.shape

(4809584,)



In [42]:
processed_dataset.to_pickle(output_file)

if debug:
    processed_dataset.to_csv(output_file + ".csv", encoding='ISO-8859-1', sep=",", doublequote=True, index=False)

NameError: name 'processed_dataset' is not defined